In [ ]:
# Definir el tamaño del tablero 
# Se declaran dos variables y se asignan valores enteros a ambas
filas = 5
columnas = 5 

# Se crea una listas de listas
# La funcion range() genera una secuencia de numeros enteros, segun el valor asignado a la variable. 
# en el bucle for, por cada numero en la secuencia generada por range() el bucle se ejecutara una vez. 
# " _ " significa que no se utilizara el valor actual del bucle (por convencion en python)
# Dentro del bucle, para cada numero en la secuencia (iteracion); se crea una cadena con un solo espacio en blanco " "
# Se toman cada uno de los espacios en blanco creados y se colocan en una nueva lista. 
# Ejemplo: columnas = 3; [" ", " ", " "]. Esto representa una unica fila del tablero. 


# Se crea el tablero bidimensional (matriz) lleno de "vacio"
tablero = [[" " for _ in range(columnas)] for _ in range(filas)] 

# Se definen los puntos de inicio para los jugadores
# Se representan con coordenadas (filas, columnas)
# Se accede a las posiciones a traves de indices, y para obtener el indice de la ultima fila, se resta 1 al numero total de filas (filas - 1)

gato_inicio = (0,0)
raton_inicio = (filas -1, columnas -1)

# Se colocan a los jugadores en sus puntos de inicio en el tablero

tablero[gato_inicio[0]][gato_inicio[1]] = "P1"
tablero[raton_inicio[0]][raton_inicio[1]] = "P2"

# Para visualizar el tablero 

def imprimir_tablero(tablero):
    for fila in tablero:
        print(" | ".join(fila)) 
        print("-" * (len(fila)*4 -1))     

print("--- Tablero Inicial ---")
imprimir_tablero(tablero)

print(f"\nEl punto de inicio del Gato es: {gato_inicio}")
print(f"El punto de inicio del Ratón es: {raton_inicio}")


## Funcion evaluar tablero
""" La funcion recibe el estado actual del tablero 
    devuelve un numero que representa que tan bien le va al gato (p1)
    1. La funcion debe saber donde estan el Gato (p1) y el raton (p2)
    2. Calcular la distancia manhattan entre ambos puntos.
    3. Definir el valor de la evaluacion:
        Mientras mas cerca este el gato del raton, el valor es mayor
        Mientras mas lejos, menor el valor 
        """

def evaluar_partida(tablero, filas, columnas):
    gato_pos = None # None representa la ausencia de un valor. Es un tipo de dato NoneType. 
    raton_pos = None

    # Encontramos la posicion actual (P1) (P2). Se recorre el tablero buscando sus símbolos.
    # Elif / Else : En cuanto se encuentra un valor, la celda ya no necesita ser revisada por las demas condiciones.
    # Con una sola pasada, identificamos la posicion de ambos jugadores
    for f in range(len(tablero)):        # Itera sobre las filas
        for c in range(len(tablero[0])): # Itera sobre las columnas
            if tablero[f][c] == "P1":
                gato_pos = (f, c)
            elif tablero[f][c] == "P2": 
                raton_pos = (f, c)

    # Si el tablero estuviera vacío, como si el raton fue "capturado" o "escapo" considerar ajustar posteriormente
    if not gato_pos or not raton_pos: 
        return 0 # Pendiente de ajuste posterior
    
    # Se calcula la distancia Manhattan entre el gato y el ratón
    distancia = abs(gato_pos[0] - raton_pos[0]) + abs(gato_pos[1] - raton_pos[1])

    # Se calcula la distancia máxima posible en el tablero (Filas y columnas ya esta parametrizado)
    distancia_max = (filas -1) + (columnas - 1)

    # Mientras menor sea la distancia, mayor es la evaluación (bueno para el gato / Maximizador)
    evaluacion = distancia_max - distancia

    # Caso de victoria del gato: Si la distancia es 0
    if distancia == 0:
        return 1000 # Un valor muy alto 

    return evaluacion



## Funcion movimientos validos
""" Se verifican y se generan posibles movimientos (aun no es el movimiento real en el tablero)
    Lista de coordenadas a las que el jugador PODRIA moverse. 
    MiniMax necesita una lista de -todos los movimientos posibles- para poder explorar el -arbol del juego-
    Por cada movimiento en la lista, Minimax: crea un tablero hipotetico/copia con el movimiento realizado.
    Llama recursivamente a si mismo para evaluar el tablero hipotetico
    1. Recibe las coordenadas actuales del jugador ((f,c))
    2. Toma en cuenta las posibles "direcciones" de movimiento.
    3. Verifica si cada posible nueva posicion esta dentro de los limites del tablero.
    No queremos que los jugadores se salgan del mapa.  
    """

def movimientos_ok(posicion, filas, columnas):
    movimientos = [] # Aqui se guardan todas las posiciones validas a las que se puede mover
    fila_actual, col_actual = posicion # Desempaquetar la posicion actual

    # Posibles direcciones de movimiento: (df, dc)
    # (-1, 0) = Arriba
    # ( 1, 0) = Abajo
    # ( 0, -1) = Izquierda
    # ( 0, 1) = Derecha
    direcciones = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    for df, dc in direcciones: 
        nueva_fila = fila_actual + df # Se calcula la nueva fila
        nueva_col = col_actual + dc # Se calcula la nueva columna

        """ Para verificar si la nueva posicion esta dentro de los limites del tablero:
        La fila debe ser >= 0 y < filas (ej. 0 a 4 para 5 filas)
        La columna debe ser >= " y < columnas *(ej. 0 a 4 para 5 columnas)"""

        if 0 <= nueva_fila < filas and 0 <= nueva_col < columnas: 
            movimientos.append((nueva_fila, nueva_col)) # Si es valida, la añadimos a la lista

    return movimientos

# Prueba de la funcion movimientos validos, para la posicion inicial del gato (eliminar despues)
gato_movs = movimientos_ok(gato_inicio, filas, columnas)
print(f"Movimientos validos para el gato desde {gato_inicio} : {gato_movs}")
raton_movs = movimientos_ok(raton_inicio, filas, columnas)
print(f"movimientos validos para el raton desde {raton_inicio} : {raton_movs}")


# Constantes del juego. 
turnos_max = 50 # Numero maximo de turnos antes de empatar 

# Funcion verificar estado del juego
def verificar_estado_juego(gato_pos, raton_pos, turno_actual, filas, columnas):
    # Condicion 1: Victoria del gato (P1)
    # Si el gato esta en la misma posicion que el ratón. 
    if gato_pos == raton_pos:
        return "Ganador: 🐈" #El gato "atrapo" al ratón. 
    
    # Condicion 2: Victoria del Ratón (P2)
    # Si el ratón ha llegado a cualquier celda en la fila superior (fila 0)
    if raton_pos[0] == 0: # r = 0 significa que el ratón llego a la fila superior
        return "Ganador: 🐀" # El ratón escapó
    
    # Condicion 3: Empate por límite de turnos
    # Si se alcanza el numero máximo de turnos y nadie ha ganado
    if turno_actual >= turnos_max: 
        return "Empate" # Límite de turnos alcanzado, ratón sobrevivió pero no escapó
    
    # Si ninguna de las condicones anteriores se cumple, el juego continúa
    return "Partida en curso"

""" Pruebas de funcion 
current_gato_pos = gato_inicio
current_raton_pos = raton_inicio

# Probar estado inicial (turno 0)
print(f"Estado del juego en Turno 0: {verificar_estado_juego(current_gato_pos, current_raton_pos, 0, filas, columnas)}")

# Simular que el gato atrapa al ratón (en este caso, ambos en (0,0))
gato_atrapa_pos = (2,2)
raton_atrapado_pos = (2,2)
print(f"Estado del juego (Gato Atrapa): {verificar_estado_juego(gato_atrapa_pos, raton_atrapado_pos, 5, filas, columnas)}")

# Simular que el ratón escaapa (ej. Ratón en (0,3))
gato_persiguiendo_pos = (1,3)
raton_escapa_pos = (0,3)
print(f"Estado del juego (Ratón escapa): {verificar_estado_juego(gato_persiguiendo_pos, raton_escapa_pos, 10, filas, columnas)}")

# Simular empate por turnos (nadie gana, turnos agotados)
gato_lejos_pos = (1,1)
raton_lejos_pos = (3,3)
print(f"Estado del juego(empate por turnos):{verificar_estado_juego(gato_lejos_pos, raton_lejos_pos, turnos_max, filas, columnas)}") """ 



--- Tablero Inicial ---
P1 |   |   |   |  
-------------------
  |   |   |   |  
-------------------
  |   |   |   |  
-------------------
  |   |   |   |  
-------------------
  |   |   |   | P2
-------------------

El punto de inicio del Gato es: (0, 0)
El punto de inicio del Ratón es: (4, 4)
Movimientos validos para el gato desde (0, 0) : [(1, 0), (0, 1)]
movimientos validos para el raton desde (4, 4) : [(3, 4), (4, 3)]


' Pruebas de funcion \ncurrent_gato_pos = gato_inicio\ncurrent_raton_pos = raton_inicio\n\n# Probar estado inicial (turno 0)\nprint(f"Estado del juego en Turno 0: {verificar_estado_juego(current_gato_pos, current_raton_pos, 0, filas, columnas)}")\n\n# Simular que el gato atrapa al ratón (en este caso, ambos en (0,0))\ngato_atrapa_pos = (2,2)\nraton_atrapado_pos = (2,2)\nprint(f"Estado del juego (Gato Atrapa): {verificar_estado_juego(gato_atrapa_pos, raton_atrapado_pos, 5, filas, columnas)}")\n\n# Simular que el ratón escaapa (ej. Ratón en (0,3))\ngato_persiguiendo_pos = (1,3)\nraton_escapa_pos = (0,3)\nprint(f"Estado del juego (Ratón escapa): {verificar_estado_juego(gato_persiguiendo_pos, raton_escapa_pos, 10, filas, columnas)}")\n\n# Simular empate por turnos (nadie gana, turnos agotados)\ngato_lejos_pos = (1,1)\nraton_lejos_pos = (3,3)\nprint(f"Estado del juego(empate por turnos):{verificar_estado_juego(gato_lejos_pos, raton_lejos_pos, turnos_max, filas, columnas)}") '

### Plan de Acción
1. Fundamentos y reglas del juego


    1.1 Definición del Tablero:
    
        * Definir el tamaño del tablero (filas, columnas). Actualmente 5x5
        
        * Crear el tablero bidimensional (matriz) inicializado con espacios en blanco
        
        --> Falta: Probar utilizar arrays
        
    1.2 Posiciones iniciales de los jugadores:

        * Definir los puntos de inicio del Gato (P1) y el Ratón (P2)

        * Colocar a los jugadores en sus posiciones iniciales en el tablero

    1.3 Funciones auxiliares esenciales:

        * imprimir_tablero(tablero): Visualiza el estado actual del tablero.

        * evaluar_partida(tablero, filas, columnas): Asigna una puntuación al estado del tablero(utilizando distancia Manhattan).

        * movimientos_ok(posicion, filas, columnas): Genera una lista de todos los movimientos validos dentro de los limites del tablero.

    1.4 Definición de reglas de juego y condiciones de fin:

        * Victoria del Gato (P1): Atrapa al ratón (misma celda)

        * Victoria del Ratón (P2): Llega a cualquier celda de la fila 0. 
        
        * Limite de turnos/empate: Si se alcanzan turnos_max y nadie gano. 

        verificar_estado_juego(gato_pos, raton_pos, turno_actual, filas, columnas): Comprueba las condiciones y devuelve el estado del juego.

2. Lógica principal del juego (Pendiente⏳)


    2.1 Función principal jugar_partida():
    
        * Inicializar el tablero y el contador de turnos.

        * Bucle principal del juego que continua hasta que verificar_estado_juego indique un fin.

        * Alternar turnos entre el Gato y el Ratón. 

    2.2 Manejo de movimientos: 

        * Para el gato (P1): Implementar lógica para que el movimiento sea manual (input)

        * Para el ratón (P2): Implementar lógica para que el movimiento sea manual (input), o para empezar un movimiento aleatorio. 

        * Validar la entrada del jugador. 

        * Actualizar el tablero despues de un movimiento. 

    2.3 Actualización y visualizacion de estado:

        * Imprimir el tablero actualizado despues de cada movimiento.

        * Mostrar el turno actual y el estado del juego (verificar_estado_juego)

        * Anuncia el resutlado final (ganador/empate)

3. Integración de Algoritmo MiniMax (Pendiente⏳)


    3.1 Implementacion:

        * Crear la funcion minimax(tablero, profundidad, es_turno_maximizador, alpha, beta)
        
        * Lógica recursiva para explorar el arbol de juego

        * Utilizar evaluar_partida y movimientos_ok para las ramificaciones. 

    3.2 Busqueda de mejor movimiento:

        * Funcion que utlizando minimax encuentre el mejor movimiento para el jugador actual.

    3.3 Reemplazo de movimentos input:

        * Integrar la funcion en jugar_partida() para que el P1 o P2 o ambos jueguen automaticamente. 